In [4]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [5]:
#Directory per Framar, Jimmy, Leo
dir=r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\Spotify_Million\data'

In [7]:
# Initialize data structures for deduplication
artists = {}
albums = {}
tracks = {}

# Helper function to get or create IDs for deduplication
def get_or_create_id(mapping, value):
    if value not in mapping:
        mapping[value] = len(mapping) + 1
    return mapping[value]

# Directory containing JSON files

# Initialize an empty list for storing playlists as rows
playlists_data = []

for filename in tqdm(os.listdir(dir)):
    if filename.endswith(".json"):
        dir_file = os.path.join(dir, filename)
        try:
            with open(dir_file, "r") as json_file:
                data = json.load(json_file)

                for playlist in data.get("playlists", []):
                    playlist_tracks = []

                    for track in playlist.get("tracks", []):
                        reduced_track = (
                            track.get("pos"),
                            #"tu": track.get("track_uri"),
                            get_or_create_id(artists, track.get("artist_name")),
                            get_or_create_id(albums, track.get("album_name")),
                            get_or_create_id(tracks, track.get("track_name")), 
                            track.get("duration_ms") // 1000
                        ) # (pos, artist_name, album_name, track_name and duration_ms)
                        playlist_tracks.append(reduced_track)
                        del reduced_track

                    # Add the playlist to the dataframe structure
                    playlists_data.append({
                        "user_id": playlist.get("pid"),
                        "name": playlist.get("name"),
                        "tracks": playlist_tracks
                    })
                    del playlist_tracks

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

with open('artists.json', 'w') as artists_file:
    json.dump(artists, artists_file, indent=4)

with open('albums.json', 'w') as albums_file:
    json.dump(albums, albums_file, indent=4)

with open('tracks.json', 'w') as tracks_file:
    json.dump(tracks, tracks_file, indent=4)

del artists
del albums
del tracks 

# Convert playlists data to a Pandas DataFrame
playlists_df = pd.DataFrame(playlists_data)

del playlists_data

# Save the DataFrame to a file
output_dir = r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System'
os.makedirs(output_dir, exist_ok=True)

# Save as CSV instead of Parquet
output_file = os.path.join(output_dir, "playlists.csv")
playlists_df.to_csv(output_file, index=False)


print(f"Processing completed. DataFrame saved to {output_file}.")


100%|██████████| 1000/1000 [09:05<00:00,  1.83it/s]


Processing completed. DataFrame saved to C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\playlists.csv.


In [9]:
playlists_df

,user_id,name,tracks
0,0,Throwbacks,"[(0, 1, 1, 1), (1, 2, 2, 2), (2, 3, 3, 3), (3,..."
1,1,Awesome Playlist,"[(0, 38, 45, 52), (1, 39, 46, 53), (2, 39, 46,..."
2,2,korean,"[(0, 59, 68, 91), (1, 60, 69, 92), (2, 61, 70,..."
3,3,mat,"[(0, 90, 119, 155), (1, 90, 120, 156), (2, 91,..."
4,4,90s,"[(0, 176, 226, 281), (1, 177, 227, 282), (2, 1..."
...,...,...,...
999995,999995,Praise,"[(0, 2287, 3711, 116657), (1, 820, 1993, 5783)..."
999996,999996,Worship,"[(0, 92000, 185884, 2260492), (1, 92000, 18588..."
999997,999997,Sex,"[(0, 1656, 2600, 39638), (1, 1606, 2574, 41138..."
999998,999998,✝️,"[(0, 638, 3672, 5768), (1, 2300, 3744, 5872), ..."
